# Nova Model Whiteboard

This code is just me experimenting with building my own neural network, none of this has been unit tested, time or memory efficiency optimized, etc...

In [1]:
import numpy as np
import tensorflow as tf
from neuralnetworks import ffnn, embedding, attention, masking, transformer, constants
import importlib

2024-07-27 21:28:01.542055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Embedding Model Tests

In [2]:
importlib.reload(embedding)

<module 'neuralnetworks.embedding' from '/Users/joericks/Desktop/nova/model/neuralnetworks/embedding.py'>

In [24]:
e_dim = constants.embd_constants.dimension

e_layer = embedding.EmbeddingLayer(e_dim, name = "embedding")

e_layer("hello")
e_layer("world")

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
        2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
       -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
        1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
        1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
        2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
        1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
       -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
        2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
       -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
       -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
       -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
       -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00,
        8.1321323e-01,  2.616

In [26]:
other_layer = embedding.EmbeddingLayer(e_dim, name = "other_embedding")

other_layer("!")

other_layer("=")

other_layer (";")

new_layer = e_layer + other_layer

In [27]:
new_layer.h

{'hello': 0, 'world': 1, '!': 2, '=': 3, ';': 4}

Documentation on softmax and ReLU are provided below: <br>
**[ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))** <br>
**[softmax](https://en.wikipedia.org/wiki/Softmax_function)**

In [28]:
#testing the performer attention head

sentence = ['hello', 'world']

idxs = [new_layer.h[s] for s in sentence if s in new_layer.h.keys()]

In [29]:
c = []

for i in idxs:
    c.append(new_layer.embeddings[i].numpy())

In [30]:
# then this process would get iterated based on the batches of training data passed through the model,
# but here we are going to cheat and just make this example be one batch

b = []

b.append(c)

batches = tf.constant(b)


In [31]:
batches

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
          2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
         -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
          1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
          1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
          2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
          1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
         -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
          2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
         -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
         -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
         -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
         -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00

## Attention Mechanism Testing

In [63]:
importlib.reload(attention)
importlib.reload(masking)

<module 'neuralnetworks.masking' from '/Users/joericks/Desktop/nova/model/neuralnetworks/masking.py'>

In [64]:
attention_mech = attention.PerformerAttention(e_dim, 8)

attention_mech(batches, batches, batches)

(1, 8, 2, 32)
(1, 4, 256)


<tf.Tensor: shape=(1, 4, 256), dtype=float32, numpy=
array([[[-2.0460699 , -0.15627328, -0.48905292, ..., -0.8268525 ,
         -2.2852986 ,  1.5125973 ],
        [ 0.21411942, -0.7596054 ,  0.10175006, ...,  0.61415154,
         -0.6894134 , -0.13938618],
        [-2.0460699 , -0.15627328, -0.48905292, ..., -0.8268525 ,
         -2.2852986 ,  1.5125973 ],
        [ 0.21411942, -0.7596054 ,  0.10175006, ...,  0.61415154,
         -0.6894134 , -0.13938618]]], dtype=float32)>

In [54]:
batches

<tf.Tensor: shape=(1, 2, 256), dtype=float32, numpy=
array([[[ 1.5942723e+00, -6.7547476e-01, -2.6637869e+00, -2.3073845e-01,
          2.3068399e+00, -1.7264534e+00, -1.0379984e+00, -1.4703115e+00,
         -2.4398067e+00,  1.0663488e+00,  4.7101808e-01, -9.8482651e-01,
          1.9035375e+00,  2.2587671e+00, -6.6063637e-01,  7.0880644e-02,
          1.2899162e+00, -8.4084612e-01, -6.9391769e-01,  1.9873365e+00,
          2.5168214e+00,  2.6040128e-01, -6.5319583e-02, -8.7325698e-01,
          1.8292906e+00, -1.7624538e-01, -1.6859952e-01,  5.8000642e-01,
         -5.1502746e-02,  6.6792262e-01, -5.1596463e-02,  1.8066067e-03,
          2.3532717e+00,  2.4322982e+00,  4.9038255e-01, -4.5623708e-01,
         -9.1632295e-01,  1.0419813e+00, -4.4167089e-01, -1.7496036e-01,
         -1.4974694e+00, -4.6423760e-01, -9.8539364e-01, -5.4634824e-02,
         -1.5379286e+00,  7.0096117e-01, -2.4192855e+00, -2.0769010e+00,
         -8.8361102e-01, -8.3318537e-01, -2.1546333e-01, -2.2244570e+00

## Transformer Testing

In [44]:
importlib.reload(transformer)

<module 'neuralnetworks.transformer' from '/Users/joericks/Desktop/nova/model/neuralnetworks/transformer.py'>

In [45]:
tfrmr = transformer.TransformerLayer(batches.shape[2], batches.shape[0], name = "test_transformer")

In [46]:
tfrmr(batches)

(2, 8, 2, 2)
(1, 8, 2, 32)


2024-07-27 21:59:23.453390: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,2,256] vs. [1,4,256]


InvalidArgumentError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [1,2,256] vs. [1,4,256] [Op:AddV2] name: 